In [1]:
import json
from datetime import datetime, date
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import os

In [2]:
def get_date(date_str):
    return datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ").date()

def convert_time_to_seconds(time_str):
    time = datetime.strptime(time_str, "%Y-%m-%dT%H:%M:%S.%fZ").time()
    return time.hour * 3600 + time.minute * 60 + time.second

In [3]:
def get_game_info(match, tournament,region,stage):
    game_info = match[0]
    game_end = match[-1]
    
    game = pd.DataFrame()
    
    start_time = convert_time_to_seconds(game_info["eventTime"])
    #print(start_time)
    end_time = convert_time_to_seconds(game_end["eventTime"])
    date = get_date(game_info["eventTime"])

    game["date"] = [date]
    game["start_time"] = [start_time]
    game["end_time"] = [end_time]
    game["duration"] = [game.iloc[0]["end_time"] - game.iloc[0]["start_time"]]

    patch = game_info["gameVersion"]
    game["patch"] =[patch]

    participants =game_info["participants"]
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    
    for player,role in zip(participants,roles):
        summonerName =player["summonerName"].split(" ")
        team_tag = summonerName.pop(0)
        player_name = " ".join(summonerName)
        if player["teamID"]==100:
            side = "blue"
        else:
            side = "red"
        game[f"{side}_team"]= [team_tag]
        game[f"{side}_{role}"]= [player_name]
    
    game["stage"]=[stage]

    if game_end["winningTeam"]== 100:
        game["result"]=[1]
    else:
        game["result"]=[0]
    return(game)

In [4]:
def get_status_updates(match,game):
    start_time = game.iloc[0]["start_time"]
    #print(start_time)
    roles = ["top","jng","mid","adc","sup","top","jng","mid","adc","sup"]
    end_time = int(convert_time_to_seconds(match[-2]["eventTime"]) - start_time)
    for event in match:
        event_time = int(convert_time_to_seconds(event["eventTime"]) - start_time)
        for time in [end_time]: #can add more times, 300 sec for 5 mins game time
            if (event["eventType"] == "stats_update") and (event_time == int(time)):
                if time == end_time:
                    time = "end"
                participants = event["participants"]
                variables =["totalGold","level"] #, "shutdownValue"
                #other stats that have been taken out "WARD_KILLED", "WARD_PLACED", "TOTAL_DAMAGE_DEALT", "TOTAL_DAMAGE_TAKEN"
                stats= ["NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE", "CHAMPIONS_KILLED","NUM_DEATHS", "ASSISTS", "VISION_SCORE", "TOTAL_DAMAGE_DEALT_TO_CHAMPIONS"]
                pings = [ "BASIC_PINGS", "COMMAND_PINGS","DANGER_PINGS","GET_BACK_PINGS","RETREAT_PINGS", "ON_MY_WAY_PINGS", "ASSIST_ME_PINGS", "ENEMY_MISSING_PINGS", "PUSH_PINGS", "ALL_IN_PINGS", "HOLD_PINGS", "BAIT_PINGS", "VISION_CLEARED_PINGS", "ENEMY_VISION_PINGS", "NEED_VISION_PINGS"]
                team_stats = ["inhibKills","towerKills","baronKills","dragonKills"]
                minions = ["MINIONS_KILLED","NEUTRAL_MINIONS_KILLED","NEUTRAL_MINIONS_KILLED_YOUR_JUNGLE","NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE"]
                
                red_pings=0
                blue_pings =0
                for player,role in zip(participants,roles):
                    if player["teamID"]==100:
                        side = "blue"
                    else:
                        side = "red"
                    for var in variables:
                        game[f"{side}_{role}_{var}_{time}"] = [player[var]]
            
                    stat_list = player["stats"]
                    cs =0
                    for stat in stat_list:
                        if stat["name"] in minions:
                            name = stat["name"]
                            cs += stat["value"]
                            
                        if stat["name"] in stats:
                            name = stat["name"]
                            game[f"{side}_{role}_{name}_{time}"] = stat["value"]
                            
                        elif stat["name"] in pings:
                            if side =="blue":
                                blue_pings += stat["value"]
                            else:
                                red_pings += stat["value"]
    
                            
                    game[f"{side}_{role}_cs_{time}"] = [cs]
                game[f"blue_pings_{time}"] = blue_pings
                game[f"red_pings_{time}"] = red_pings
                for stat in team_stats:
                    game[f"blue_{stat}_{time}"] =[event["teams"][0][stat]]
                    game[f"red_{stat}_{time}"] =[event["teams"][1][stat]]
    return(game)

In [5]:
#dictionary for team regions
regions = {}
df = pd.DataFrame()
directory = "C:/GLOBAL POWER RANKINGS/games/"
for folder in os.listdir(directory):
    tournament = folder.split("_")[0]
    if tournament != "msi" or tournament !="worlds" or tournament !="midseason":
        #setting the region for teams who play in the tournaments (excluding international events)
        region = tournament
    else:
        region = None
    for stage in os.listdir(f"{directory}{folder}"):
        print(f" processing {directory}{folder}/{stage}")
        for games in os.listdir(f"{directory}{folder}/{stage}"):
            with open(f"{directory}{folder}/{stage}/{games}", "r") as json_file:
                match = json.load(json_file)
                game = get_game_info(match,tournament,region,stage)
                #print(game)
                game = get_status_updates(match,game)
                df =pd.concat([df,game])
                
df.head()

 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2022/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2022/Regular Season
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Groups
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_spring_2023/Regular Season
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2022/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2022/Regular Season
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2023/Groups
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2023/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_summer_2023/Regular Season
 processing C:/GLOBAL POWER RANKINGS/games/lec_winter_2023/Groups
 processing C:/GLOBAL POWER RANKINGS/games/lec_winter_2023/Playoffs
 processing C:/GLOBAL POWER RANKINGS/games/lec_winter_2023/Regular Season


,date,start_time,end_time,duration,patch,blue_team,blue_top,blue_jng,blue_mid,blue_adc,...,blue_pings_end,red_pings_end,blue_inhibKills_end,red_inhibKills_end,blue_towerKills_end,red_towerKills_end,blue_baronKills_end,red_baronKills_end,blue_dragonKills_end,red_dragonKills_end
0,2022-03-25,61956,64249,2293,12.5.428.2127,RGE,Odoamne,Malrang,Larssen,Comp,...,0,0,2,0,9,3,2,0,3,3
0,2022-03-25,71591,72965,1374,12.5.428.2127,RGE,Odoamne,Malrang,Larssen,Comp,...,0,0,1,0,9,1,0,0,1,0
0,2022-03-27,54491,56737,2246,12.5.428.2127,XL,Finn,Markoon,Nukeduck,Patrik,...,0,0,2,0,11,4,2,0,4,2
0,2022-03-27,58002,59880,1878,12.5.428.2127,VIT,Alphari,Selfmade,Perkz,Carzzy,...,0,0,1,0,8,5,1,0,4,0
0,2022-03-27,61141,63428,2287,12.5.428.2127,XL,Finn,Markoon,Nukeduck,Patrik,...,0,0,1,1,7,7,0,1,2,2


In [6]:
#df = df.sort_values("start_time")
df = df.sort_values(by =["date", "start_time"])
df.reset_index(drop=True, inplace=True)
df_og =df

In [23]:
df =df_og

In [24]:
#df.head(244)

,date,start_time,end_time,duration,patch,blue_team,blue_top,blue_jng,blue_mid,blue_adc,...,red_sup_ASSISTS_end_rolling,red_sup_VISION_SCORE_end_rolling,red_sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,red_sup_cs_end_rolling,red_pings_end_rolling,red_inhibKills_end_rolling,red_towerKills_end_rolling,red_baronKills_end_rolling,red_dragonKills_end_rolling,red_duration_rolling
0,2022-01-14,61466,63404,1938,12.1.416.5961,MAD,Armut,Elyoya,Reeker,UNF0RGIVEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-01-14,64745,66413,1668,12.1.416.5961,RGE,Odoamne,Malrang,Larssen,Comp,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2022-01-14,68211,70738,2527,12.1.416.5961,XL,Finn,Markoon,Nukeduck,Patrik,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2022-01-14,72200,74851,2651,12.1.416.5961,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2022-01-14,75995,78091,2096,12.1.416.5961,FNC,Wunder,Razork,Humanoid,Upset,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239,2022-09-11,62215,63977,1762,12.15.458.1416,RGE,Odoamne,Malrang,Larssen,Comp,...,5.333333,83.053051,6482.271647,25.533333,132.666667,0.666667,5.333333,0.333333,2.000000,2018.666667
240,2023-01-21,62846,65280,2434,13.1.487.8994,XL,Odoamne,Xerxe,Vetheo,Patrik,...,1.666667,80.897915,5951.489258,25.866667,119.333333,0.333333,3.333333,0.000000,1.333333,2085.666667
241,2023-01-21,67623,70037,2414,13.1.487.8994,KOI,Szygenda,Malrang,Larssen,Comp,...,9.666667,91.152949,6005.805501,36.600000,342.333333,0.333333,4.000000,0.333333,1.666667,1935.666667
242,2023-01-21,72132,74381,2249,13.1.487.8994,AST,Finn,113,Dajor,Kobbe,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df.to_csv("lec_spring_2023_test.csv",index=True)

In [10]:
blue_cols = ["date","start_time", "duration"] +[col for col in df.columns if "blue" in col]
red_cols = ["date","start_time", "duration"] +[col for col in df.columns if "red" in col]
team_cols =[col.replace("blue_","") for col in blue_cols]
#print(team_cols)
blue = df[blue_cols]
red = df[red_cols]

In [11]:
#Get the tags of all of the teams
teams = df["blue_team"].drop_duplicates()
#make rge = koi etc

In [12]:
#separating the data for individual teams to get rolling averages
team_data = pd.DataFrame()
for team in teams:
    for index,row in df.iterrows():
        if row["blue_team"] == team:
            blue_renamed = blue.rename(columns=dict(zip(blue_cols, team_cols)))
            team_data = pd.concat([team_data, blue_renamed.iloc[[index]]])
        elif row["red_team"] == team:
            red_renamed = red.rename(columns=dict(zip(red_cols, team_cols)))
            team_data = pd.concat([team_data, red_renamed.iloc[[index]]])

In [13]:
team_data.to_csv("lec_spring_2023_teamdata.csv",index=True)

In [14]:
grouped_games = team_data.groupby("team")

In [15]:
def rolling_avg(group,cols,new_cols):
    group = group.sort_values("date")
    rolling_stats = group[cols].rolling(3, closed ='left').mean() #assigns avg from previous 3 games onto the 4th game
    group[new_cols]=rolling_stats
    group = group.dropna(subset = new_cols) #drops the first 3 games without rolling averages
    return group

In [16]:
#need to change what cols it uses for rolling avg(instead of just _end ones)
cols = [col for col in team_data.columns if "_end" in col] + ["duration"]
new_cols = [f"{c}_rolling" for c in cols]

In [17]:
games_rolling = grouped_games.apply(lambda x:rolling_avg(x,cols,new_cols))
#games_rolling

In [18]:
games_rolling = games_rolling.droplevel('team')
games_rolling

,date,start_time,duration,team,top,jng,mid,adc,sup,top_totalGold_end,...,sup_ASSISTS_end_rolling,sup_VISION_SCORE_end_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,sup_cs_end_rolling,pings_end_rolling,inhibKills_end_rolling,towerKills_end_rolling,baronKills_end_rolling,dragonKills_end_rolling,duration_rolling
16,2022-01-21,64371,2513,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,13255,...,6.000000,94.994380,1950.931559,38.000000,0.000000,0.000000,1.666667,0.333333,1.666667,2077.333333
21,2022-01-22,61094,1939,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,10918,...,4.333333,98.205393,3625.774089,41.333333,0.000000,0.000000,2.000000,0.000000,1.333333,2031.333333
25,2022-01-28,61172,2036,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,10185,...,6.333333,97.074005,5025.667969,30.333333,0.000000,0.000000,3.000000,0.333333,1.333333,1981.000000
31,2022-01-29,60568,2154,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,14842,...,6.666667,112.187589,6253.532389,34.000000,0.000000,0.000000,3.000000,0.333333,1.000000,2162.666667
35,2022-02-04,61081,1864,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,12002,...,6.666667,106.783702,5681.764323,27.333333,0.000000,0.000000,2.666667,0.666667,1.000000,2043.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488,2023-07-29,61120,1822,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,15483,...,11.000000,111.042727,5727.228678,33.000000,506.333333,1.333333,7.666667,1.333333,3.333333,2005.000000
489,2023-07-29,67816,1702,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,13446,...,13.000000,102.200490,5086.724935,32.666667,510.666667,1.333333,8.000000,1.333333,3.666667,1957.000000
491,2023-07-30,61989,2234,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,13654,...,16.333333,84.162295,4720.623698,33.533333,429.000000,1.666667,9.000000,1.333333,3.000000,1751.000000
490,2023-07-30,58780,1938,XL,Odoamne,Peach,Abbedagge,Patrik,LIMIT,8236,...,14.000000,131.077077,5962.141439,40.533333,409.000000,1.000000,7.333333,1.333333,2.333333,1919.333333


In [19]:
games = games_rolling.sort_index()
games.head()

,date,start_time,duration,team,top,jng,mid,adc,sup,top_totalGold_end,...,sup_ASSISTS_end_rolling,sup_VISION_SCORE_end_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,sup_cs_end_rolling,pings_end_rolling,inhibKills_end_rolling,towerKills_end_rolling,baronKills_end_rolling,dragonKills_end_rolling,duration_rolling
15,2022-01-21,61102,1914,MSF,HiRit,Shlatan,Vetheo,Neon,Mersa,13334,...,8.000000,87.427465,4643.912842,23.333333,0.0,1.333333,7.666667,1.333333,2.666667,2227.666667
15,2022-01-21,61102,1914,XL,Finn,Markoon,Nukeduck,Patrik,Advienne,12116,...,10.000000,113.557388,4265.861247,31.750000,0.0,1.000000,7.333333,0.666667,2.666667,2355.333333
16,2022-01-21,64371,2513,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,13255,...,6.000000,94.994380,1950.931559,38.000000,0.0,0.000000,1.666667,0.333333,1.666667,2077.333333
16,2022-01-21,64371,2513,FNC,Wunder,Razork,Humanoid,Upset,Hylissang,16990,...,11.000000,107.750394,4048.891927,33.666667,0.0,1.666667,8.333333,1.000000,3.000000,2051.000000
17,2022-01-21,68188,1945,G2,BrokenBlade,Jankos,caPs,Flakked,Targamas,12851,...,10.666667,117.489616,4803.900065,42.750000,0.0,1.000000,6.666667,1.000000,3.333333,2121.000000


In [22]:
games.head(455)

,date,start_time,duration,team,top,jng,mid,adc,sup,top_totalGold_end,...,sup_ASSISTS_end_rolling,sup_VISION_SCORE_end_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,sup_cs_end_rolling,pings_end_rolling,inhibKills_end_rolling,towerKills_end_rolling,baronKills_end_rolling,dragonKills_end_rolling,duration_rolling
15,2022-01-21,61102,1914,MSF,HiRit,Shlatan,Vetheo,Neon,Mersa,13334,...,8.000000,87.427465,4643.912842,23.333333,0.000000,1.333333,7.666667,1.333333,2.666667,2227.666667
15,2022-01-21,61102,1914,XL,Finn,Markoon,Nukeduck,Patrik,Advienne,12116,...,10.000000,113.557388,4265.861247,31.750000,0.000000,1.000000,7.333333,0.666667,2.666667,2355.333333
16,2022-01-21,64371,2513,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,promisq,13255,...,6.000000,94.994380,1950.931559,38.000000,0.000000,0.000000,1.666667,0.333333,1.666667,2077.333333
16,2022-01-21,64371,2513,FNC,Wunder,Razork,Humanoid,Upset,Hylissang,16990,...,11.000000,107.750394,4048.891927,33.666667,0.000000,1.666667,8.333333,1.000000,3.000000,2051.000000
17,2022-01-21,68188,1945,G2,BrokenBlade,Jankos,caPs,Flakked,Targamas,12851,...,10.666667,117.489616,4803.900065,42.750000,0.000000,1.000000,6.666667,1.000000,3.333333,2121.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
240,2023-01-21,62846,2434,XL,Odoamne,Xerxe,Vetheo,Patrik,Targamas,10574,...,14.666667,101.934853,9534.319499,12.333333,414.000000,1.666667,8.000000,1.000000,3.000000,2281.333333
240,2023-01-21,62846,2434,G2,BrokenBlade,Yike,Caps,Hans Sama,Mikyx,12788,...,1.666667,80.897915,5951.489258,25.866667,119.333333,0.333333,3.333333,0.000000,1.333333,2085.666667
241,2023-01-21,67623,2414,BDS,Adam,Sheo,nuc,Crownie,Labrov,12431,...,9.666667,91.152949,6005.805501,36.600000,342.333333,0.333333,4.000000,0.333333,1.666667,1935.666667
242,2023-01-21,72132,2249,AST,Finn,113,Dajor,Kobbe,JeongHoon,12564,...,6.666667,123.188166,6434.970215,33.666667,575.666667,0.000000,4.000000,0.000000,1.333333,2258.666667


In [24]:
# Recombining separate game data into the main set
rolling_cols = [col for col in games.columns if "rolling" in col]
blue_rolling_cols = [f"blue_{col}" for col in rolling_cols]
red_rolling_cols = [f"red_{col}" for col in rolling_cols]
#print(red_rolling_cols)
result_df = pd.DataFrame()
for team in teams:
    for index, row in df.iterrows():
        try:
            if index in games.index and row["blue_team"] == team:
                # Copy the original game DataFrame and rename columns
                game = games.rename(columns=dict(zip(rolling_cols, blue_rolling_cols)))
                # Adding in the rolling columns
                game = game[(game.index == index) & (game["team"] == team)]
                #print(game.index)
                new_cols = game.loc[index,blue_rolling_cols]
                #print(new_cols)
                #df = pd.concat([df, new_cols], axis=1)
                #result_df = pd.concat([result_df, new_cols], axis=1)
                #result_df[blue_rolling_cols] = new_cols
                df.loc[index, blue_rolling_cols]= new_cols
            elif index in games.index and row["red_team"] == team:
                # Copy the original game DataFrame and rename columns
                game = games.rename(columns=dict(zip(rolling_cols, red_rolling_cols)))
                game = game[(game.index == index) & (game["team"] == team)]
                game
                new_cols = game.loc[index,red_rolling_cols]
                #df = pd.concat([df, new_cols], axis=1)
                #result_df = pd.concat([result_df, new_cols], axis=1)
                #result_df[red_rolling_cols] = new_cols
                df.loc[index, red_rolling_cols]= new_cols
        except KeyError:
            #its skipping the first 3 games for "new" teams as this data has been removed
            print(f"{index} for {team} not found")
            continue

241 for KOI not found
248 for KOI not found
253 for KOI not found
242 for TH not found
247 for TH not found
250 for TH not found


In [25]:
df = df.dropna()
df.head()

,date,start_time,end_time,duration,patch,blue_team,blue_top,blue_jng,blue_mid,blue_adc,...,red_sup_ASSISTS_end_rolling,red_sup_VISION_SCORE_end_rolling,red_sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,red_sup_cs_end_rolling,red_pings_end_rolling,red_inhibKills_end_rolling,red_towerKills_end_rolling,red_baronKills_end_rolling,red_dragonKills_end_rolling,red_duration_rolling
15,2022-01-21,61102,63016,1914,12.1.418.7728,MSF,HiRit,Shlatan,Vetheo,Neon,...,10.000000,113.557388,4265.861247,31.750000,0.0,1.000000,7.333333,0.666667,2.666667,2355.333333
16,2022-01-21,64371,66884,2513,12.1.418.7728,AST,WhiteKnight,Zanzarah,Dajor,Kobbe,...,11.000000,107.750394,4048.891927,33.666667,0.0,1.666667,8.333333,1.000000,3.000000,2051.000000
17,2022-01-21,68188,70133,1945,12.1.418.7728,G2,BrokenBlade,Jankos,caPs,Flakked,...,9.333333,99.802699,8893.754720,27.416667,0.0,1.000000,4.000000,0.666667,3.000000,2332.333333
18,2022-01-21,71501,73178,1677,12.1.418.7728,BDS,Adam,Cinkrof,NUCLEARINT,xMatty,...,6.000000,91.627490,3606.544515,31.000000,0.0,0.000000,2.333333,0.000000,1.333333,2083.333333
19,2022-01-21,74715,76803,2088,12.1.418.7728,MAD,Armut,Elyoya,Reeker,UNF0RGIVEN,...,10.000000,72.911303,3863.059652,28.850000,0.0,1.666667,9.333333,1.000000,2.000000,1767.000000


In [26]:
#Converting categorical variables into numerical
roles = ["top","jng","mid","adc","sup"]
for side in ["blue","red"]:
    for role in roles:
        df[f"{side}_{role}_code"] =  df[f"{side}_{role}"].astype("category").cat.codes
    df[f"{side}_team_code"] = df[f"{side}_team"].astype("category").cat.codes

In [27]:
df.to_csv("lec_spring_2023.csv",index=False)

In [154]:
code_cols = [col for col in df.columns if "_code" in col]
rolling_cols =[col for col in df.columns if "rolling" in col]
features = ["start_time"] + code_cols + rolling_cols
print(features)

['start_time', 'blue_top_code', 'blue_jng_code', 'blue_mid_code', 'blue_adc_code', 'blue_sup_code', 'blue_team_code', 'red_top_code', 'red_jng_code', 'red_mid_code', 'red_adc_code', 'red_sup_code', 'red_team_code', 'red_top_totalGold_end_rolling', 'red_top_level_end_rolling', 'red_top_NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE_end_rolling', 'red_top_CHAMPIONS_KILLED_end_rolling', 'red_top_NUM_DEATHS_end_rolling', 'red_top_ASSISTS_end_rolling', 'red_top_VISION_SCORE_end_rolling', 'red_top_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling', 'red_top_cs_end_rolling', 'red_jng_totalGold_end_rolling', 'red_jng_level_end_rolling', 'red_jng_NEUTRAL_MINIONS_KILLED_ENEMY_JUNGLE_end_rolling', 'red_jng_CHAMPIONS_KILLED_end_rolling', 'red_jng_NUM_DEATHS_end_rolling', 'red_jng_ASSISTS_end_rolling', 'red_jng_VISION_SCORE_end_rolling', 'red_jng_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling', 'red_jng_cs_end_rolling', 'red_mid_totalGold_end_rolling', 'red_mid_level_end_rolling', 'red_mid_NEUTRAL_MINIONS_KILLED_ENEMY

In [52]:
X = games[features]
y = games["result"]

In [53]:
X

,side_code,start_time,team_code,top_code,jng_code,mid_code,adc_code,sup_code,enemy_top_code,enemy_jng_code,...,sup_VISION_SCORE_end_rolling,sup_TOTAL_DAMAGE_DEALT_end_rolling,sup_TOTAL_DAMAGE_DEALT_TO_CHAMPIONS_end_rolling,sup_TOTAL_DAMAGE_TAKEN_end_rolling,inhibKills_end_rolling,towerKills_end_rolling,baronKills_end_rolling,dragonKills_end_rolling,total_pings_rolling,duration_rolling
0,0,61709,0,4,0,3,6,3,2,2,...,99.860382,16794.774740,4427.193197,12957.264160,0.666667,5.666667,0.666667,1.666667,630.333333,1728.000000
0,1,61709,5,2,2,5,0,2,4,0,...,72.915736,17991.160807,5415.185791,15793.098958,0.333333,4.666667,0.333333,2.000000,439.666667,1821.333333
0,1,75931,3,1,9,1,4,8,9,4,...,89.894414,25134.458333,9303.239746,11583.531250,1.000000,7.333333,0.666667,3.333333,387.000000,1658.333333
0,1,72355,6,5,5,8,3,1,8,1,...,94.017652,17294.555990,4520.162760,9673.050456,1.000000,8.333333,1.000000,3.000000,521.333333,1881.666667
0,0,68295,2,7,6,2,8,0,0,7,...,81.961739,16381.268880,7043.188477,7767.103516,0.000000,2.000000,0.000000,1.000000,240.333333,1767.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1,65139,1,0,7,10,2,6,2,2,...,82.417299,18332.043294,4220.047526,13524.517904,1.000000,5.666667,0.666667,2.666667,578.666667,1972.666667
0,0,70823,1,0,7,10,2,6,2,2,...,101.628176,19194.934245,4268.654785,11593.568034,1.333333,7.666667,1.666667,3.666667,662.333333,2187.666667
0,1,61837,1,0,7,10,2,6,2,2,...,88.095445,17717.075195,3267.629883,10769.328125,1.000000,6.666667,1.000000,2.666667,580.000000,1970.333333
0,0,61837,5,2,2,5,0,2,0,7,...,76.946331,18592.972331,2829.449300,12599.928060,2.333333,7.000000,1.000000,2.333333,521.000000,1784.666667
